<a href="https://colab.research.google.com/github/daniel3400-ui/DoctorPatientQueryAI/blob/main/DLCoursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dan3400","key":"0a1e5e54fcb0ac505f4d77e3e0409cd2"}'}

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Adjust permissions

In [ ]:
!kaggle datasets download -d dsxavier/diagnoise-me

 98% 187M/191M [00:05<00:00, 45.3MB/s]
100% 191M/191M [00:05<00:00, 38.6MB/s]


In [ ]:
!unzip diagnoise-me.zip

Archive:  diagnoise-me.zip
  inflating: diagnose_en_dataset.feather  
  inflating: en_medical_dialog.json  


In [ ]:
import pandas as pd

df = pd.read_feather('/content/diagnose_en_dataset.feather')
df.head()

,id,Description,Doctor,Patient
0,0,Q. What does abutment of the nerve root mean?,Hi. I have gone through your query with dilige...,"Hi doctor,I am just wondering what is abutting..."
1,1,"Q. Every time I eat spicy food, I poop blood. ...",Hello. I have gone through your information an...,"Hi doctor, I am a 26 year old male. I am 5 fee..."
2,2,Q. Will Nano-Leo give permanent solution for e...,Hi. For further doubts consult a sexologist on...,"Hello doctor, I am 48 years old. I am experien..."
3,3,Q. Will Kalarchikai cure multiple ovarian cyst...,Hello. I just read your query. See Kalarachi K...,"Hello doctor, I have multiple small cysts in b..."
4,4,Q. I masturbate only by rubbing the tip of the...,Hi. For further doubts consult a sexologist on...,"Hi doctor, During masturbation I just rub the ..."


In [ ]:
missing_values = df.isnull().sum()
print(missing_values)

id             0
Description    0
Doctor         0
Patient        0
dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257469 entries, 0 to 257468
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           257469 non-null  int64 
 1   Description  257469 non-null  object
 2   Doctor       257469 non-null  object
 3   Patient      257469 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.9+ MB


In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    if text.startswith('q '):
        text = text[2:]
    return text

df['Description'] = df['Description'].apply(clean_text)
df['Doctor'] = df['Doctor'].apply(clean_text)
df['Patient'] = df['Patient'].apply(clean_text)

print(df.head())

   id                                        Description  \
0   0          what does abutment of the nerve root mean   
1   1       every time i eat spicy food i poop blood why   
2   2  will nanoleo give permanent solution for erect...   
3   3  will kalarchikai cure multiple ovarian cysts i...   
4   4  i masturbate only by rubbing the tip of the pe...   

                                              Doctor  \
0  hi i have gone through your query with diligen...   
1  hello i have gone through your information and...   
2  hi for further doubts consult a sexologist onl...   
3  hello i just read your query see kalarachi kai...   
4  hi for further doubts consult a sexologist onl...   

                                             Patient  
0  hi doctori am just wondering what is abutting ...  
1  hi doctor i am a 26 year old male i am 5 feet ...  
2  hello doctor i am 48 years old i am experienci...  
3  hello doctor i have multiple small cysts in bo...  
4  hi doctor during masturb

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset
from sklearn.model_selection import train_test_split

df['Context'] = "Description: " + df['Description'] + " " + "Question: " + df['Patient']
df['Response'] = "Doctor: " + df['Doctor']

train_df, val_df = train_test_split(df[['Context', 'Response']], test_size=0.1)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)


In [ ]:
!

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:

tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_function(examples):
    model_inputs = tokenizer(examples['Context'], max_length=250, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['Response'], max_length=250, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

Map:   0%|          | 0/231722 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/25747 [00:00<?, ? examples/s]

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import os
import shutil
import torch


model = T5ForConditionalGeneration.from_pretrained('t5-small').cuda()

output_dir = './results'

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)


training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.01,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    fp16=True,
    weight_decay=0.01,
    save_total_limit=3,
    warmup_steps=500,
    num_train_epochs=3,
    predict_with_generate=True,
    max_grad_norm=1.0,
    lr_scheduler_type="linear",
    load_best_model_at_end=True
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)


trainer.train()


Epoch,Training Loss,Validation Loss
0,1.255100,1.131162
2,1.076700,1.015797


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=5430, training_loss=1.2203298326355319, metrics={'train_runtime': 4006.589, 'train_samples_per_second': 173.506, 'train_steps_per_second': 1.355, 'total_flos': 4.5928716140544e+16, 'train_loss': 1.2203298326355319, 'epoch': 2.9991714995857497})

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch

def generate_response(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)  #

    prompt = "question: " + text
    inputs = tokenizer(prompt, return_tensors="pt", max_length=500, truncation=True, padding="max_length")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(inputs['input_ids'], max_length=500, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    response= tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


patient_query = "I've been dealing with a severe headache since this morning that started suddenly. It's mostly on the right side of my head and gets worse when I move. I haven't experienced anything like this before. Could this be a migraine, or should I be worried about something more serious?"
patient_query = re.sub(r'[^\w\s]', '', patient_query)
patient_query = patient_query.lower()

print(generate_response(patient_query))


Doctor: hello and welcome to ask a doctor service i have reviewed your query and here is my advice the headache that you are experiencing is likely to be due to migraine you need to get an mri brain done to rule out any intracranial pathology meanwhile you can take acetaminophen for pain relief if symptoms not improved please consult your doctor he will examine and treat you accordingly take care


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.save_pretrained('/content/drive/My Drive/Model')
